# Data collection and preprocessing

In [2]:
import os
import pandas as pd
import json
import uuid
import re
import nltk
import spacy
import gensim
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora
from transformers import BartTokenizer
from openai import OpenAI

In [3]:

data = []

# Open and read the JSON Lines file
with open('datasets/Cell_Phones_and_Accessories_5.json', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append it to the list
        data.append(json.loads(line))


print(data[:5])

[{'reviewerID': 'A30TL5EWN6DFXT', 'asin': '120401325X', 'reviewerName': 'christina', 'helpful': [0, 0], 'reviewText': "They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again", 'overall': 4.0, 'summary': 'Looks Good', 'unixReviewTime': 1400630400, 'reviewTime': '05 21, 2014'}, {'reviewerID': 'ASY55RVNIL0UD', 'asin': '120401325X', 'reviewerName': 'emily l.', 'helpful': [0, 0], 'reviewText': 'These stickers work like the review says they do. They stick on great and they stay on the phone. They are super stylish and I can share them with my sister. :)', 'overall': 5.0, 'summary': 'Really great product.', 'unixReviewTime': 1389657600, 'reviewTime': '01 14, 2014'}, {'reviewerID': 'A2TMXE2AFO7ONB', 'asin': '120401325X', 'reviewerName': 'Erica', 'helpful': [0, 0], 'reviewText': 'These are awesome and make my phone look so stylish! I have only used one so far an

In [4]:
amazon_reviews = pd.DataFrame(data)


amazon_reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4.0,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5.0,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [5]:
amazon_reviews.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [6]:
amazon_reviews.shape

(194439, 9)

In [7]:
sephora_reviews = pd.read_csv("datasets/reviews_0-250.csv", low_memory=False)

sephora_reviews.head()

,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
3,3,6083038851,5,1.0,NaN,0,0,0,2023-03-20,I’ve always loved this formula for a long time...,Can't go wrong with any of them,NaN,brown,combination,black,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
4,4,47056667835,5,1.0,NaN,0,0,0,2023-03-20,"If you have dry cracked lips, this is a must h...",A must have !!!,light,hazel,combination,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0


In [8]:
sephora_reviews.shape

(602130, 19)

In [9]:
sephora_reviews.columns

Index(['Unnamed: 0', 'author_id', 'rating', 'is_recommended', 'helpfulness',
       'total_feedback_count', 'total_neg_feedback_count',
       'total_pos_feedback_count', 'submission_time', 'review_text',
       'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color',
       'product_id', 'product_name', 'brand_name', 'price_usd'],
      dtype='object')

In [10]:
clothing_reviews = pd.read_csv("datasets/Womens Clothing E-Commerce Reviews.csv")
clothing_reviews.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [11]:
clothing_reviews.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

In [12]:
clothing_reviews.shape

(23486, 11)

## Organizing and storing data into one file

In [13]:
# renaming the columns to match new dataframe of all the combined dataset
amazon_reviews = amazon_reviews.rename(columns={
    'reviewerID': 'review_id',
    'reviewText': 'review_text',
    'overall': 'rating',
    'asin': 'product_id',
    'summary': 'review_title',
    'reviewTime': 'review_date'
})

In [14]:
# add product category column
amazon_reviews['product_category'] = 'cellphones and accessories'

# add column to show source of review
amazon_reviews['source'] = 'Amazon'

In [15]:
amazon_reviews = amazon_reviews[['review_id', 'review_text','review_title', 'rating', 'product_id', 'review_date', 'product_category', 'source']]

In [16]:
# check for missing values
amazon_reviews.isnull().sum()

review_id           0
review_text         0
review_title        0
rating              0
product_id          0
review_date         0
product_category    0
source              0
dtype: int64

In [17]:
amazon_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194439 entries, 0 to 194438
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   review_id         194439 non-null  object 
 1   review_text       194439 non-null  object 
 2   review_title      194439 non-null  object 
 3   rating            194439 non-null  float64
 4   product_id        194439 non-null  object 
 5   review_date       194439 non-null  object 
 6   product_category  194439 non-null  object 
 7   source            194439 non-null  object 
dtypes: float64(1), object(7)
memory usage: 11.9+ MB


In [18]:
amazon_reviews['review_date'].head()

0    05 21, 2014
1    01 14, 2014
2    06 26, 2014
3    10 21, 2013
4     02 3, 2013
Name: review_date, dtype: object

In [19]:
# Convert amazon_reviews['review_date'] with a specified format

amazon_reviews.loc[:, 'review_date'] = pd.to_datetime(amazon_reviews['review_date'], format="%m %d, %Y", errors='coerce')

In [20]:
amazon_reviews['review_date'].head()

0    2014-05-21 00:00:00
1    2014-01-14 00:00:00
2    2014-06-26 00:00:00
3    2013-10-21 00:00:00
4    2013-02-03 00:00:00
Name: review_date, dtype: object

In [21]:
amazon_reviews['review_date'].isna().sum()

0

In [22]:
# renaming the columns to match new dataframe of all the combined dataset
sephora_reviews = sephora_reviews.rename(columns={
    'author_id': 'review_id',
    'review_text': 'review_text',
    'rating': 'rating',
    'submission_time': 'review_date',
    'product_name': 'product_name'

})

In [23]:
# add new categories
sephora_reviews['product_category'] = 'skincare'

sephora_reviews['source'] = 'Sephora'

In [24]:
# Select target columns
sephora_reviews = sephora_reviews[['review_id', 'review_text','review_title', 'rating', 'product_id', 'review_date', 'product_category', 'source']]


In [25]:
sephora_reviews['review_date'].head()

0    2023-02-01
1    2023-03-21
2    2023-03-21
3    2023-03-20
4    2023-03-20
Name: review_date, dtype: object

In [26]:
# e-commerce clothing store
clothing_reviews = clothing_reviews.rename(columns={
    'Clothing ID': 'product_id',
    'Review Text': 'review_text',
    'Rating': 'rating',
    'Department Name': 'product_category',
    'Title': 'review_title'
})

In [27]:
# add new categories
clothing_reviews['source'] = 'E-commerce Clothing Store'


In [28]:
# Generate a unique review_id for each review
clothing_reviews['review_id'] = [str(uuid.uuid4()) for _ in range(len(clothing_reviews))]

In [29]:
# Add a fixed review_date
fixed_review_date = '2019-05-23'

# add review date column
clothing_reviews['review_date'] = fixed_review_date

In [30]:
clothing_reviews['review_date'].head()

0    2019-05-23
1    2019-05-23
2    2019-05-23
3    2019-05-23
4    2019-05-23
Name: review_date, dtype: object

In [31]:
# Select target columns
clothing_reviews = clothing_reviews[['review_id', 'review_text','review_title', 'rating', 'product_id', 'review_date', 'product_category', 'source']]

In [32]:
clothing_reviews.head()

,review_id,review_text,review_title,rating,product_id,review_date,product_category,source
0,92fccc46-4984-4bae-ba8e-007a27f511be,Absolutely wonderful - silky and sexy and comf...,NaN,4,767,2019-05-23,Intimate,E-commerce Clothing Store
1,991406e4-6cb7-46e8-abfd-a4c8ed805c58,Love this dress! it's sooo pretty. i happene...,NaN,5,1080,2019-05-23,Dresses,E-commerce Clothing Store
2,290a819a-7bb8-4ec4-86a2-a7be2e25725f,I had such high hopes for this dress and reall...,Some major design flaws,3,1077,2019-05-23,Dresses,E-commerce Clothing Store
3,5e04fb91-fad0-4cc5-9f8d-854cb4878c20,"I love, love, love this jumpsuit. it's fun, fl...",My favorite buy!,5,1049,2019-05-23,Bottoms,E-commerce Clothing Store
4,a1b8413c-f6f9-4538-b438-d8eaf1c149f4,This shirt is very flattering to all due to th...,Flattering shirt,5,847,2019-05-23,Tops,E-commerce Clothing Store


In [33]:
# combine all three datasets

combined_reviews = pd.concat([amazon_reviews, sephora_reviews, clothing_reviews], ignore_index=True)

In [34]:
combined_reviews.head()

,review_id,review_text,review_title,rating,product_id,review_date,product_category,source
0,A30TL5EWN6DFXT,They look good and stick good! I just don't li...,Looks Good,4.0,120401325X,2014-05-21 00:00:00,cellphones and accessories,Amazon
1,ASY55RVNIL0UD,These stickers work like the review says they ...,Really great product.,5.0,120401325X,2014-01-14 00:00:00,cellphones and accessories,Amazon
2,A2TMXE2AFO7ONB,These are awesome and make my phone look so st...,LOVE LOVE LOVE,5.0,120401325X,2014-06-26 00:00:00,cellphones and accessories,Amazon
3,AWJ0WZQYMYFQ4,Item arrived in great time and was in perfect ...,Cute!,4.0,120401325X,2013-10-21 00:00:00,cellphones and accessories,Amazon
4,ATX7CZYFXI1KW,"awesome! stays on, and looks great. can be use...",leopard home button sticker for iphone 4s,5.0,120401325X,2013-02-03 00:00:00,cellphones and accessories,Amazon


In [35]:
combined_reviews.shape

(820055, 8)

# Data cleaning

In [36]:
# check data types
combined_reviews.columns

Index(['review_id', 'review_text', 'review_title', 'rating', 'product_id',
       'review_date', 'product_category', 'source'],
      dtype='object')

In [37]:
# change review date data type to datetime
combined_reviews['review_date'] = pd.to_datetime(combined_reviews['review_date'])

In [38]:
# check for missing values
combined_reviews.isnull().sum()

review_id                0
review_text           1844
review_title        170821
rating                   0
product_id               0
review_date              0
product_category        14
source                   0
dtype: int64

In [39]:
# drop all missing values
combined_reviews = combined_reviews.dropna(subset=['review_text', 'review_title', 'product_category'])

combined_reviews .isna().sum()

review_id           0
review_text         0
review_title        0
rating              0
product_id          0
review_date         0
product_category    0
source              0
dtype: int64

In [40]:
combined_reviews.shape

(649220, 8)

In [41]:
combined_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 649220 entries, 0 to 820054
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   review_id         649220 non-null  object        
 1   review_text       649220 non-null  object        
 2   review_title      649220 non-null  object        
 3   rating            649220 non-null  float64       
 4   product_id        649220 non-null  object        
 5   review_date       649220 non-null  datetime64[ns]
 6   product_category  649220 non-null  object        
 7   source            649220 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 44.6+ MB


## Save the new dataset

In [42]:
#combined_reviews.to_csv("curated_reviews_data.csv", index=False)

In [43]:
# Define data types for each column
dtype_dict = {
    "review_id": "string",
    "review_text": "string",
    "review_title": "string",
    "rating": "float64",
    "product_id": "string",
    "product_category": "string",
    "source": "string"
}

data = pd.read_csv('curated_reviews_data.csv', dtype=dtype_dict, parse_dates=['review_date'])


# Pre-processing

## Text preprocessing data using NLTK

In [44]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/chijofareen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/chijofareen/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chijofareen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/chijofareen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [45]:
combined_reviews['full_text'] = combined_reviews['review_title'] + ". " + combined_reviews['review_text']

In [46]:
combined_reviews['full_text'] 

0         Looks Good. They look good and stick good! I j...
1         Really great product.. These stickers work lik...
2         LOVE LOVE LOVE. These are awesome and make my ...
3         Cute!. Item arrived in great time and was in p...
4         leopard home button sticker for iphone 4s. awe...
                                ...                        
820050    Great dress for many occasions. I was very hap...
820051    Wish it was made of cotton. It reminds me of m...
820052    Cute, but see through. This fit well, but the ...
820053    Very cute dress, perfect for summer parties an...
820054    Please make more like this one!. This dress in...
Name: full_text, Length: 649220, dtype: object

## Theme detection & Sentiment analysis

In [47]:
# Preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return ''  
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    tokens = word_tokenize(text)  # Tokenize
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return ' '.join(tokens)

In [48]:
# Apply preprocessing function to the reviews
data['clean_review_text'] = data['review_text'].apply(preprocess_text)
data['clean_review_title'] = data['review_title'].apply(preprocess_text)

In [49]:
# Show pre-processed review
data[['clean_review_text', 'clean_review_title']].head()

,clean_review_text,clean_review_title
0,look good stick good dont like rounded shape a...,looks good
1,stickers work like review says stick great sta...,really great product
2,awesome make phone look stylish used one far a...,love love love
3,item arrived great time perfect condition howe...,cute
4,awesome stays looks great used multiple apple ...,leopard home button sticker iphone


In [50]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

In [51]:
# Combine preprocessed text for analysis
data['full_text'] = (
    data['clean_review_title'] + " " + data['clean_review_text']
)

# Sample subset of data 
sample_size = int(0.3 * len(combined_reviews))  # Adjust fraction as needed
subset_data = data.sample(n=sample_size, random_state=42)[['full_text']].copy()

In [52]:

# Display the size of the subset for verification
print(f"Sampled {len(subset_data)} rows for topic modeling and sentiment analysis.")

Sampled 194766 rows for topic modeling and sentiment analysis.


In [53]:
# Sentiment Analysis with VADER
sia = SentimentIntensityAnalyzer()

subset_data['sentiments'] = subset_data['full_text'].apply(lambda x: sia.polarity_scores(x))
subset_data['compound'] = subset_data['sentiments'].apply(lambda score_dict: score_dict['compound'])
subset_data['sentiment_type'] = subset_data['compound'].apply(lambda c: 'positive' if c > 0 else ('negative' if c < 0 else 'neutral'))


In [54]:
# Preprocess text for LDA
def preprocess_for_lda(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\S*@\S*\s?', '', text)  # Remove emails
    text = re.sub(r"\'", '', text)  # Remove apostrophes
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove non-alphabet characters
    return text.lower()

subset_data['cleaned_text'] = subset_data['full_text'].apply(preprocess_for_lda)

### Tokenize and Remove Stopwords

In [55]:
# Tokenize and remove stopwords
stop_words = set(stopwords.words('english'))
def tokenize(text):
    tokens = gensim.utils.simple_preprocess(text, deacc=True)
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

subset_data['tokens'] = subset_data['cleaned_text'].apply(tokenize)

### Lemmatize Tokens

In [56]:
# Lemmatize using spaCy
def lemmatize(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

subset_data['lemmas'] = subset_data['tokens'].apply(lemmatize)

### Create Dictionary and Corpus, Build LDA Model and print topics

In [57]:
# Create dictionary and corpus for LDA
id2word = corpora.Dictionary(subset_data['lemmas'])
texts = subset_data['lemmas']
corpus = [id2word.doc2bow(text) for text in texts]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=16, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)



In [58]:
topics = lda_model.show_topics(num_topics=16, num_words=10, formatted=False)
topic_keywords = {f"Topic {i}": [word for word, _ in words] for i, words in topics}
topic_keywords

{'Topic 0': ['charge',
  'battery',
  'charger',
  'device',
  'cable',
  'power',
  'usb',
  'car',
  'plug',
  'phone'],
 'Topic 1': ['phone',
  'screen',
  'easy',
  'quality',
  'protector',
  'easily',
  'protect',
  'hold',
  'iphone',
  'samsung'],
 'Topic 2': ['acne',
  'clear',
  'breakout',
  'spot',
  'area',
  'dark',
  'cause',
  'prone',
  'huge',
  'redness'],
 'Topic 3': ['great',
  'use',
  'get',
  'one',
  'go',
  'buy',
  'time',
  'little',
  'even',
  'try'],
 'Topic 4': ['worth',
  'line',
  'money',
  'live',
  'spend',
  'happen',
  'offer',
  'waste',
  'person',
  'dead'],
 'Topic 5': ['skin',
  'product',
  'love',
  'use',
  'feel',
  'face',
  'dry',
  'moisturizer',
  'smell',
  'makeup'],
 'Topic 6': ['cream',
  'eye',
  'receive',
  'sample',
  'free',
  'quickly',
  'influenster',
  'absorb',
  'complimentary',
  'honest'],
 'Topic 7': ['lip',
  'balm',
  'wish',
  'type',
  'next',
  'thin',
  'base',
  'pay',
  'honestly',
  'pair'],
 'Topic 8': ['cl

# Tagging themes 

### Assign theme to reviews

In [59]:
subset_data.head()

,full_text,sentiments,compound,sentiment_type,cleaned_text,tokens,lemmas
479857,skin better love wanted hate price keeper does...,"{'neg': 0.219, 'neu': 0.54, 'pos': 0.24, 'comp...",0.2964,positive,skin better love wanted hate price keeper does...,"[skin, better, love, wanted, hate, price, keep...","[skin, well, love, want, hate, price, keeper, ..."
458777,read im gon na honest expecting little wowed f...,"{'neg': 0.059, 'neu': 0.563, 'pos': 0.378, 'co...",0.9648,positive,read im gon na honest expecting little wowed f...,"[read, im, gon, na, honest, expecting, little,...","[read, I, m, go, to, honest, expect, little, w..."
81597,love love long cord cant stand short ones like...,"{'neg': 0.0, 'neu': 0.456, 'pos': 0.544, 'comp...",0.9643,positive,love love long cord cant stand short ones like...,"[love, love, long, cord, cant, stand, short, o...","[love, love, long, cord, can, not, stand, shor..."
596330,good smell received product complimentary farm...,"{'neg': 0.129, 'neu': 0.636, 'pos': 0.235, 'co...",0.8771,positive,good smell received product complimentary farm...,"[good, smell, received, product, complimentary...","[good, smell, receive, product, complimentary,..."
105916,pretty pretty dust plugger took long time get ...,"{'neg': 0.257, 'neu': 0.338, 'pos': 0.405, 'co...",0.5423,positive,pretty pretty dust plugger took long time get ...,"[pretty, pretty, dust, plugger, took, long, ti...","[pretty, pretty, dust, plugger, take, long, ti..."


In [60]:
# Function to assign theme based on dominant topic
def assign_refined_theme(distribution, topic_to_theme_map):
    # Gets the topic with the highest probability
    dominant_topic = max(distribution, key=lambda x: x[1])[0]
    # Map the dominant topic to the refined theme
    return topic_to_theme_map.get(f"Topic {dominant_topic}", "Unknown")

# Map the LDA topics to refined themes
topic_to_theme_map = {
    'Topic 0': 'Product Performance & Features',
    'Topic 1': 'Phone Protection & Customization',
    'Topic 2': 'Skin Care Journey & Acne Solutions',
    'Topic 3': 'Customer Satisfaction',
    'Topic 4': 'Value & Affordability',
    'Topic 5': 'Skincare Effectiveness',
    'Topic 6': 'Skincare Ingredients & Moisturization',
    'Topic 7': 'Product Recommendations',
    'Topic 8': 'Customer Satisfaction',
    'Topic 9': 'Customer Satisfaction',
    'Topic 10': 'Technology & Sound Quality',
    'Topic 11': 'Phone Protection & Customization',
    'Topic 12': 'Phone Protection & Customization',
    'Topic 13': 'Product Recommendations',
    'Topic 14': 'Customer Satisfaction',
    'Topic 15': 'Skincare Effectiveness'
}

# Assign themes based on topic distributions
topic_distributions = [lda_model.get_document_topics(doc) for doc in corpus]
subset_data['assigned_theme'] = [assign_refined_theme(dist, topic_to_theme_map) for dist in topic_distributions]

In [61]:
subset_data[['full_text', 'assigned_theme']].head()

,full_text,assigned_theme
479857,skin better love wanted hate price keeper does...,Customer Satisfaction
458777,read im gon na honest expecting little wowed f...,Skincare Effectiveness
81597,love love long cord cant stand short ones like...,Customer Satisfaction
596330,good smell received product complimentary farm...,Customer Satisfaction
105916,pretty pretty dust plugger took long time get ...,Customer Satisfaction


## Tokenization with Hugging Face Transformers

In [62]:
# Load the pre-trained BART tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Tokenize the 'full_text' for summarization
def tokenize_function(examples):
    return tokenizer(examples['full_text'], padding='max_length', truncation=True, max_length=512)

# Tokenizing the 'full_text' in subset_data
subset_data['tokenized'] = subset_data['full_text'].apply(lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=512))

In [63]:
# The resulting tokenized data will include the input IDs and attention masks
subset_data['input_ids'] = subset_data['tokenized'].apply(lambda x: x['input_ids'])
subset_data['attention_mask'] = subset_data['tokenized'].apply(lambda x: x['attention_mask'])

subset_data[['full_text', 'input_ids', 'attention_mask']].head()

,full_text,input_ids,attention_mask
479857,skin better love wanted hate price keeper does...,"[0, 33986, 357, 657, 770, 4157, 425, 13456, 47...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
458777,read im gon na honest expecting little wowed f...,"[0, 12745, 4356, 38197, 2750, 5322, 4804, 410,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
81597,love love long cord cant stand short ones like...,"[0, 17693, 657, 251, 13051, 17672, 1413, 765, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
596330,good smell received product complimentary farm...,"[0, 8396, 11362, 829, 1152, 22310, 3380, 5073,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
105916,pretty pretty dust plugger took long time get ...,"[0, 28674, 1256, 8402, 10242, 2403, 362, 251, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [64]:
# Save the processed subset_data to a CSV file
#subset_data.to_csv('processed_subset_data.csv', index=False)

#print("Processed data saved as 'processed_subset_data.csv'")


Processed data saved as 'processed_subset_data.csv'
